In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from deform_conv import DeformConv2D
from torch.utils.data import Dataset, DataLoader
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 
from PIL import Image

In [ ]:
import sys
print('[INFO]    Reading training data...')
X = np.loadtxt('./MNIST/comp551w18-modified-mnist/train_x.csv', delimiter=',')
print('[INFO]    Reading training labels...')
Y = np.loadtxt('./MNIST/comp551w18-modified-mnist/train_y.csv', delimiter=',')

In [ ]:
X = X.reshape(-1, 64, 64) # reshape 
Y = Y.reshape(-1, 1) 
Image.fromarray(np.uint8(X[100])*255, mode='L')

In [ ]:
X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, test_size=0.2)
training_set = (torch.from_numpy(X_train), torch.from_numpy(Y_train))
test_set = (torch.from_numpy(X_valid), torch.from_numpy(Y_valid))

In [ ]:
print('[INFO]    Writing Torch data files...')
torch.save(training_set, './MNIST/comp551w18-modified-mnist/training.pt')
torch.save(test_set, './MNIST/comp551w18-modified-mnist/test.pt')

In [ ]:
del training_set
del test_set
del X_train
del X_valid
del Y_train
del Y_valid
del X
del Y

In [2]:
class Args():  
    batch_size = 32
    test_batch_size = 32
    epochs = 1
    lr = 0.01
    momentum = 0.5
    no_cuda = False
    cuda = True
    seed = 1
    log_interval = 1 
args = Args()

In [3]:
args.cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}

In [4]:
class DeformNet(nn.Module):
    def __init__(self):
        super(DeformNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)

        self.offsets = nn.Conv2d(128, 18, kernel_size=3, padding=1)
        self.conv4 = DeformConv2D(128, 128, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(128)

        self.classifier = nn.Linear(128, 10)

    def forward(self, x):
        # convs
        x = F.relu(self.conv1(x))
        x = self.bn1(x)
        x = F.relu(self.conv2(x))
        x = self.bn2(x)
        x = F.relu(self.conv3(x))
        x = self.bn3(x)
        # deformable convolution
        offsets = self.offsets(x)
        x = F.relu(self.conv4(x, offsets))
        x = self.bn4(x)

        x = F.avg_pool2d(x, kernel_size=28, stride=1).view(x.size(0), -1)
        x = self.classifier(x)

        return F.log_softmax(x, dim=1)

model = DeformNet()

In [5]:
def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        nn.init.xavier_uniform(m.weight, gain=nn.init.calculate_gain('relu'))
        if m.bias is not None:
            m.bias.data = torch.FloatTensor(m.bias.shape[0]).zero_()


def init_conv_offset(m):
    m.weight.data = torch.zeros_like(m.weight.data)
    if m.bias is not None:
        m.bias.data = torch.FloatTensor(m.bias.shape[0]).zero_()


model.apply(init_weights)
model.offsets.apply(init_conv_offset)

Conv2d(128, 18, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [6]:
if args.cuda:
    model.cuda()

In [7]:
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

In [34]:
import sys
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target.squeeze().type(torch.cuda.LongTensor))
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if args.cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [9]:
import errno
class MMNISTTrainTestDataset(Dataset):
    
    training_file = 'training.pt'
    test_file = 'test.pt'

    def __init__(self, root_dir, train=True, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.train = train
        self.root = root_dir
        self.transform = transform
        if self.train:
            self.train_data, self.train_labels = torch.load(
                os.path.join(self.root, self.training_file))
            self.train_data = self.train_data / 255.
        else:
            self.test_data, self.test_labels = torch.load(
                os.path.join(self.root, self.test_file))
            self.test_data = self.test_data / 255.
        

        
        
    def _check_exists(self):
        return os.path.exists(os.path.join(self.root, self.processed_folder, self.training_file)) and \
            os.path.exists(os.path.join(self.root, self.processed_folder, self.test_file))
        
    def __len__(self):
        if self.train:
            return len(self.train_data)
        else:
            return len(self.test_data)

    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
            tuple: (image, target) where target is index of the target class.
        """
        if self.train:
            img, target = self.train_data[index], self.train_labels[index]
        else:
            img, target = self.test_data[index], self.test_labels[index]

        # doing this so that it is consistent with all other datasets
        # to return a PIL Image
        img = Image.fromarray(np.uint8(img.numpy()*255), mode='L')

        if self.transform is not None:
            img = self.transform(img)

        return img, target

In [11]:
train_loader = torch.utils.data.DataLoader(
    MMNISTTrainTestDataset('./MNIST/comp551w18-modified-mnist/', train=True,
                   transform=transforms.Compose([
                       transforms.Resize((28, 28)),
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    MMNISTTrainTestDataset('./MNIST/comp551w18-modified-mnist/', train=False,
                   transform=transforms.Compose([
                       transforms.Resize((28, 28)),
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)

In [ ]:
from time import time
for epoch in range(1, args.epochs + 1):
    since = time()
    train(epoch)
    iter = time() - since
    print("Spends {}s for each training epoch".format(iter/args.epochs))
    test()

Train Epoch: 1 [0/40000 (0%)]	Loss: 2.323546
Train Epoch: 1 [32/40000 (0%)]	Loss: 2.380867
Train Epoch: 1 [64/40000 (0%)]	Loss: 2.254897
Train Epoch: 1 [96/40000 (0%)]	Loss: 2.354798
Train Epoch: 1 [128/40000 (0%)]	Loss: 2.299983
Train Epoch: 1 [160/40000 (0%)]	Loss: 2.269060
Train Epoch: 1 [192/40000 (0%)]	Loss: 2.327000
Train Epoch: 1 [224/40000 (1%)]	Loss: 2.270867
Train Epoch: 1 [256/40000 (1%)]	Loss: 2.317577
Train Epoch: 1 [288/40000 (1%)]	Loss: 2.284436
Train Epoch: 1 [320/40000 (1%)]	Loss: 2.323929
Train Epoch: 1 [352/40000 (1%)]	Loss: 2.261913
Train Epoch: 1 [384/40000 (1%)]	Loss: 2.260961
Train Epoch: 1 [416/40000 (1%)]	Loss: 2.356730
Train Epoch: 1 [448/40000 (1%)]	Loss: 2.313484
Train Epoch: 1 [480/40000 (1%)]	Loss: 2.314381
Train Epoch: 1 [512/40000 (1%)]	Loss: 2.314804
Train Epoch: 1 [544/40000 (1%)]	Loss: 2.283796
Train Epoch: 1 [576/40000 (1%)]	Loss: 2.270976
Train Epoch: 1 [608/40000 (2%)]	Loss: 2.266225
Train Epoch: 1 [640/40000 (2%)]	Loss: 2.293729
Train Epoch: 1 [67

Train Epoch: 1 [5472/40000 (14%)]	Loss: 2.318435
Train Epoch: 1 [5504/40000 (14%)]	Loss: 2.194871
Train Epoch: 1 [5536/40000 (14%)]	Loss: 2.270976
Train Epoch: 1 [5568/40000 (14%)]	Loss: 2.298033
Train Epoch: 1 [5600/40000 (14%)]	Loss: 2.275693
Train Epoch: 1 [5632/40000 (14%)]	Loss: 2.282101
Train Epoch: 1 [5664/40000 (14%)]	Loss: 2.222183
Train Epoch: 1 [5696/40000 (14%)]	Loss: 2.174795
Train Epoch: 1 [5728/40000 (14%)]	Loss: 2.237059
Train Epoch: 1 [5760/40000 (14%)]	Loss: 2.306379
Train Epoch: 1 [5792/40000 (14%)]	Loss: 2.305994
Train Epoch: 1 [5824/40000 (15%)]	Loss: 2.226216
Train Epoch: 1 [5856/40000 (15%)]	Loss: 2.294952
Train Epoch: 1 [5888/40000 (15%)]	Loss: 2.206500
Train Epoch: 1 [5920/40000 (15%)]	Loss: 2.276001
Train Epoch: 1 [5952/40000 (15%)]	Loss: 2.129086
Train Epoch: 1 [5984/40000 (15%)]	Loss: 2.280252
Train Epoch: 1 [6016/40000 (15%)]	Loss: 2.161281
Train Epoch: 1 [6048/40000 (15%)]	Loss: 2.185961
Train Epoch: 1 [6080/40000 (15%)]	Loss: 2.334401
Train Epoch: 1 [6112

Train Epoch: 1 [10848/40000 (27%)]	Loss: 2.220528
Train Epoch: 1 [10880/40000 (27%)]	Loss: 2.230414
Train Epoch: 1 [10912/40000 (27%)]	Loss: 2.099958
Train Epoch: 1 [10944/40000 (27%)]	Loss: 2.196195
Train Epoch: 1 [10976/40000 (27%)]	Loss: 2.142411
Train Epoch: 1 [11008/40000 (28%)]	Loss: 2.112108
Train Epoch: 1 [11040/40000 (28%)]	Loss: 2.171066
Train Epoch: 1 [11072/40000 (28%)]	Loss: 2.252877
Train Epoch: 1 [11104/40000 (28%)]	Loss: 2.135695
Train Epoch: 1 [11136/40000 (28%)]	Loss: 2.177778
Train Epoch: 1 [11168/40000 (28%)]	Loss: 2.215227
Train Epoch: 1 [11200/40000 (28%)]	Loss: 2.195759
Train Epoch: 1 [11232/40000 (28%)]	Loss: 2.186305
Train Epoch: 1 [11264/40000 (28%)]	Loss: 2.109537
Train Epoch: 1 [11296/40000 (28%)]	Loss: 2.193891
Train Epoch: 1 [11328/40000 (28%)]	Loss: 2.280373
Train Epoch: 1 [11360/40000 (28%)]	Loss: 2.266387
Train Epoch: 1 [11392/40000 (28%)]	Loss: 2.166569
Train Epoch: 1 [11424/40000 (29%)]	Loss: 2.168452
Train Epoch: 1 [11456/40000 (29%)]	Loss: 2.222115


Train Epoch: 1 [16128/40000 (40%)]	Loss: 2.091369
Train Epoch: 1 [16160/40000 (40%)]	Loss: 2.135465
Train Epoch: 1 [16192/40000 (40%)]	Loss: 2.103979
Train Epoch: 1 [16224/40000 (41%)]	Loss: 2.101690
Train Epoch: 1 [16256/40000 (41%)]	Loss: 2.125750
Train Epoch: 1 [16288/40000 (41%)]	Loss: 2.066510
Train Epoch: 1 [16320/40000 (41%)]	Loss: 2.060921
Train Epoch: 1 [16352/40000 (41%)]	Loss: 2.096715
Train Epoch: 1 [16384/40000 (41%)]	Loss: 2.125126
Train Epoch: 1 [16416/40000 (41%)]	Loss: 2.026484
Train Epoch: 1 [16448/40000 (41%)]	Loss: 2.181245
Train Epoch: 1 [16480/40000 (41%)]	Loss: 2.059583
Train Epoch: 1 [16512/40000 (41%)]	Loss: 2.019465
Train Epoch: 1 [16544/40000 (41%)]	Loss: 2.057662
Train Epoch: 1 [16576/40000 (41%)]	Loss: 2.035289
Train Epoch: 1 [16608/40000 (42%)]	Loss: 2.020959
Train Epoch: 1 [16640/40000 (42%)]	Loss: 2.071891
Train Epoch: 1 [16672/40000 (42%)]	Loss: 2.046439
Train Epoch: 1 [16704/40000 (42%)]	Loss: 2.102705
Train Epoch: 1 [16736/40000 (42%)]	Loss: 2.109294
